In [46]:
import pandas as pd
import numpy as np
import fastf1
import datetime
fastf1.Cache.enable_cache('cache') 

In [47]:
def find_fastest_laps(laps, cols, index_cols, lap_num):
    
    laps = laps.sort_values('LapTime')
    fastest = laps.groupby('DriverNumber').nth(0).reset_index()

    fastest_time = {}
    added_cols = []
    
    
    for col in cols:
        fastest_time[col] = laps[col].min()
        
        added_cols.append(col[:-4] + 'Percent')
        added_cols.append(col[:-4] + 'Position')
        # fastest[col[:-4] + 'Percent'] = (fastest[col] - fastest_time) / fastest_time
        # total_cols.append(col[:-4] + 'Percent')
    
    # dummies = pd.get_dummies(fastest['Compound'])
    # #dummies.columns = dummies.columns + '_1'
    # fastest = pd.concat([fastest, dummies], axis = 1)
    
    fastest = fastest[index_cols]
    

    # top_cols = ['Sector1Time', 'Sector2Time', 'Sector3Time']
    # fastest = fastest.merge(laps.groupby('DriverNumber').min()[top_cols].reset_index(), on=['DriverNumber'], how = 'outer')
    for i in range(0,lap_num):
        laps_i = laps.groupby('DriverNumber').nth(i).reset_index()

        for col in ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']:
            laps_i[col[:-4] + 'Percent'] = (laps_i[col] - fastest_time[col]) / fastest_time[col]
            laps_i[col[:-4] + 'Position'] = laps_i.sort_values(col).index
        # laps_i['LapPercent'] = (laps_i['LapTime'] - fastest_lap) / fastest_lap
        fastest = fastest.merge(laps_i[index_cols + cols + added_cols], on=index_cols, how = 'outer', suffixes=('', '_' + str(i)))
    return fastest

In [48]:
def load_practice(event, type, year):
    
    practice1 = fastf1.get_session(year, event, 'Practice 1')
    practice1.load(laps=True)
    try:
        laps1 = practice1.laps
        cols = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']
        index_cols = ['DriverNumber', 'Team', 'Driver']
        practice1 = find_fastest_laps(laps1, cols, index_cols, 2)
        practice2 = practice1
        if type == 'conventional':
            practice2 = fastf1.get_session(year, event, 'Practice 2')
            practice2.load(laps=True)
            laps2 = practice2.laps
            practice2 = find_fastest_laps(laps2, cols, index_cols, 2)

        practice_data = practice1.merge(practice2, on= index_cols, how = 'outer', suffixes=('_1', '_2'))
        return practice_data
    except:
         return pd.DataFrame()
   
def load_qualifying(event, year):
        qualifying = fastf1.get_session(year, event, 'Q')
        qualifying.load()
        try:
            results = qualifying.results

            results['FastestLap'] = results[['Q1', 'Q2', 'Q3']].apply(lambda x: min(x), axis = 1)

            fastest_lap_q = results['FastestLap'].min()
            results['LapPercent'] = (results['FastestLap'] - fastest_lap_q)/ fastest_lap_q
            return results[['DriverNumber', 'Position', 'LapPercent']]
        except:
             return pd.DataFrame()


In [49]:
years = [2018, 2019,2020, 2021, 2022, 2023]
# years = [2021, 2022, 2023]

data = pd.DataFrame()
#prev_qualifying_data = pd.DataFrame()
for year in years:
    schedule = fastf1.get_event_schedule(year)
    for index, row in schedule.iterrows():
        if row['EventFormat'] in ['conventional']:
            practice_data = load_practice(row['EventName'], row['EventFormat'], year)
            qualifying_data = load_qualifying(row['EventName'], year)
            if 'DriverNumber' in practice_data.columns and 'DriverNumber' in qualifying_data.columns:
                #if 'DriverNumber' in prev_qualifying_data.columns:
                    # qualifying_data_full = qualifying_data.merge(prev_qualifying_data, on = 'DriverNumber', suffixes=('_curr', '_prev'))
                full_data = practice_data.merge(qualifying_data, on='DriverNumber')
                for col in ['RoundNumber', 'Country', 'Location', 'OfficialEventName', 'EventDate','EventName', 'EventFormat']:
                    full_data[col] = row[col]
                data = pd.concat([data, full_data], axis = 0, ignore_index=True)
                # prev_qualifying_data = qualifying_data

core           INFO 	Loading data for Australian Grand Prix - Practice 1 [v2.3.1]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df,

In [50]:
data = pd.DataFrame(data)
prev_range = 3

In [51]:
for i in range(prev_range):
    data['PrevLapPercent' + str(i+1)] = data.groupby('DriverNumber')['LapPercent'].shift(i+1)
    data['PrevPosition' + str(i+1)] = data.groupby('DriverNumber')['Position'].shift(i+1)

In [52]:
data

,DriverNumber,Team,Driver,LapTime_1,Sector1Time_1,Sector2Time_1,Sector3Time_1,LapPercent_1,LapPosition_1,Sector1Percent_1,...,OfficialEventName,EventDate,EventName,EventFormat,PrevLapPercent1,PrevPosition1,PrevLapPercent2,PrevPosition2,PrevLapPercent3,PrevPosition3
0,10,Toro Rosso,GAS,0 days 00:01:26.494000,0 days 00:00:28.639000,0 days 00:00:23.343000,0 days 00:00:34.512000,0.029372,13.0,0.034683,...,FORMULA 1 2018 ROLEX AUSTRALIAN GRAND PRIX,2018-03-25 18:10:00,Australian Grand Prix,conventional,NaN,NaN,NaN,NaN,NaN,NaN
1,11,Force India,PER,0 days 00:01:26.767000,0 days 00:00:28.731000,0 days 00:00:23.505000,0 days 00:00:34.531000,0.032621,17.0,0.038007,...,FORMULA 1 2018 ROLEX AUSTRALIAN GRAND PRIX,2018-03-25 18:10:00,Australian Grand Prix,conventional,NaN,NaN,NaN,NaN,NaN,NaN
2,14,McLaren,ALO,0 days 00:01:25.896000,0 days 00:00:28.485000,0 days 00:00:23.312000,0 days 00:00:34.099000,0.022255,11.0,0.029120,...,FORMULA 1 2018 ROLEX AUSTRALIAN GRAND PRIX,2018-03-25 18:10:00,Australian Grand Prix,conventional,NaN,NaN,NaN,NaN,NaN,NaN
3,16,Sauber,LEC,0 days 00:01:28.853000,0 days 00:00:29.384000,0 days 00:00:24.050000,0 days 00:00:35.419000,0.057447,16.0,0.061599,...,FORMULA 1 2018 ROLEX AUSTRALIAN GRAND PRIX,2018-03-25 18:10:00,Australian Grand Prix,conventional,NaN,NaN,NaN,NaN,NaN,NaN
4,18,Williams,STR,0 days 00:01:26.636000,0 days 00:00:28.452000,0 days 00:00:23.620000,0 days 00:00:34.564000,0.031062,14.0,0.027927,...,FORMULA 1 2018 ROLEX AUSTRALIAN GRAND PRIX,2018-03-25 18:10:00,Australian Grand Prix,conventional,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,81,McLaren,PIA,0 days 00:01:21.496000,0 days 00:00:22.832000,0 days 00:00:26.606000,0 days 00:00:32.058000,0.035159,9.0,0.056890,...,FORMULA 1 PIRELLI GRAND PRIX DU CANADA 2023,2023-06-18 16:00:00,Canadian Grand Prix,conventional,0.010889,10.0,0.014433,11.0,0.019237,19.0
2055,10,Alpine,GAS,NaT,NaT,NaT,NaT,NaN,NaN,NaN,...,FORMULA 1 PIRELLI GRAND PRIX DU CANADA 2023,2023-06-18 16:00:00,Canadian Grand Prix,conventional,0.007527,4.0,0.007959,7.0,0.009192,5.0
2056,31,Alpine,OCO,NaT,NaT,NaT,NaT,NaN,NaN,NaN,...,FORMULA 1 PIRELLI GRAND PRIX DU CANADA 2023,2023-06-18 16:00:00,Canadian Grand Prix,conventional,0.010087,7.0,0.002634,4.0,0.007257,8.0
2057,44,Mercedes,HAM,NaT,NaT,NaT,NaT,NaN,NaN,NaN,...,FORMULA 1 PIRELLI GRAND PRIX DU CANADA 2023,2023-06-18 16:00:00,Canadian Grand Prix,conventional,0.007555,5.0,0.005044,6.0,0.011887,13.0


In [53]:
# one_hot = ['Compound']

# for col in data.columns:
#     for name in one_hot:
#         if name in col:
#             dummies = pd.get_dummies(data[col])
#             dummies.columns = col + '_' + dummies.columns
#             data = pd.concat([data, dummies], axis = 1)

In [54]:
data.to_pickle('train_data.pkl')

In [30]:
sched_2023 = fastf1.get_event_schedule(2023)
round_number = 5

In [31]:
pred_event = sched_2023[sched_2023['RoundNumber'] == round_number]

In [42]:
pred_data = load_practice(pred_event['EventName'].iloc[0], pred_event['EventFormat'].iloc[0], 2023)
for col in ['RoundNumber', 'Country', 'Location', 'OfficialEventName', 'EventDate','EventName', 'EventFormat']:
    pred_data[col] = pred_event[col].iloc[0]
for i in range(prev_range):
    prev_event = sched_2023[sched_2023['RoundNumber'] == round_number - i - 1]
    prev_data = load_qualifying(prev_event['EventName'].iloc[0], 2023)
    prev_data['PrevLapPercent' + str(i+1)] = prev_data['LapPercent']
    prev_data['PrevPosition' + str(i+1)] = prev_data['Position']
    pred_data = pred_data.merge(prev_data[['DriverNumber', 'PrevLapPercent' + str(i+1),'PrevPosition' + str(i+1)]], on = 'DriverNumber')

core           INFO 	Loading data for Miami Grand Prix - Practice 1 [v2.3.1]
api            INFO 	Using cached data for driver_info


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1492: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for key, value in row.iteritems():
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductio

In [43]:
pred_data

,DriverNumber,Team,Driver,LapTime_1,Sector1Time_1,Sector2Time_1,Sector3Time_1,LapPercent_1,LapPosition_1,Sector1Percent_1,...,OfficialEventName,EventDate,EventName,EventFormat,PrevLapPercent1,PrevPosition1,PrevLapPercent2,PrevPosition2,PrevLapPercent3,PrevPosition3
0,1,Red Bull Racing,VER,0 days 00:01:30.549000,0 days 00:00:30.560000,0 days 00:00:34.497000,0 days 00:00:25.492000,0.004705,17,0.001606,...,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,2023-05-07 17:30:00,Miami Grand Prix,conventional,0.001876,2.0,0.000000,1.0,0.005619,15.0
1,10,Alpine,GAS,0 days 00:01:31.104000,0 days 00:00:30.644000,0 days 00:00:34.681000,0 days 00:00:25.779000,0.010863,15,0.004359,...,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,2023-05-07 17:30:00,Miami Grand Prix,conventional,0.046406,19.0,0.010973,9.0,0.012372,10.0
2,11,Red Bull Racing,PER,0 days 00:01:31.566000,0 days 00:00:30.910000,0 days 00:00:35.030000,0 days 00:00:25.626000,0.015989,4,0.013077,...,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,2023-05-07 17:30:00,Miami Grand Prix,conventional,0.002914,3.0,NaN,20.0,0.000000,1.0
3,14,Aston Martin,ALO,0 days 00:01:31.231000,0 days 00:00:30.894000,0 days 00:00:34.807000,0 days 00:00:25.530000,0.012272,0,0.012553,...,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,2023-05-07 17:30:00,Miami Grand Prix,conventional,0.010479,6.0,0.005304,4.0,0.005268,3.0
4,16,Ferrari,LEC,0 days 00:01:30.449000,0 days 00:00:30.593000,0 days 00:00:34.262000,0 days 00:00:25.594000,0.003595,16,0.002688,...,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,2023-05-07 17:30:00,Miami Grand Prix,conventional,0.000000,1.0,0.008302,7.0,0.001756,2.0
5,18,Aston Martin,STR,0 days 00:01:31.337000,0 days 00:00:30.922000,0 days 00:00:34.669000,0 days 00:00:25.746000,0.013448,1,0.013471,...,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,2023-05-07 17:30:00,Miami Grand Prix,conventional,0.013702,9.0,0.007507,6.0,0.007704,6.0
6,2,Williams,SAR,0 days 00:01:32.619000,0 days 00:00:31.564000,0 days 00:00:35.323000,0 days 00:00:25.732000,0.027673,3,0.034512,...,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,2023-05-07 17:30:00,Miami Grand Prix,conventional,0.020349,15.0,0.023784,18.0,0.455956,20.0
7,20,Haas F1 Team,MAG,0 days 00:01:31.853000,0 days 00:00:31.072000,0 days 00:00:34.954000,0 days 00:00:25.827000,0.019173,5,0.018387,...,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,2023-05-07 17:30:00,Miami Grand Prix,conventional,0.032075,18.0,0.018206,14.0,0.015510,13.0
8,21,AlphaTauri,DEV,0 days 00:01:34.637000,0 days 00:00:32.678000,0 days 00:00:35.925000,0 days 00:00:26.034000,0.050064,12,0.071024,...,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,2023-05-07 17:30:00,Miami Grand Prix,conventional,NaN,20.0,0.020891,15.0,0.022421,18.0
9,22,AlphaTauri,TSU,0 days 00:01:32.169000,0 days 00:00:31.373000,0 days 00:00:34.992000,0 days 00:00:25.804000,0.022680,13,0.028252,...,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,2023-05-07 17:30:00,Miami Grand Prix,conventional,0.013632,8.0,0.017815,12.0,0.018966,16.0


In [12]:
# one_hot = ['Compound']

# for col in pred_data.columns:
#     for name in one_hot:
#         if name in col:
#             dummies = pd.get_dummies(pred_data[col])
#             dummies.columns = col + '_' + dummies.columns
#             pred_data = pd.concat([pred_data, dummies], axis = 1)

In [44]:
pred_data.columns

Index(['DriverNumber', 'Team', 'Driver', 'LapTime_1', 'Sector1Time_1',
       'Sector2Time_1', 'Sector3Time_1', 'LapPercent_1', 'LapPosition_1',
       'Sector1Percent_1', 'Sector1Position_1', 'Sector2Percent_1',
       'Sector2Position_1', 'Sector3Percent_1', 'Sector3Position_1',
       'LapTime_1_1', 'Sector1Time_1_1', 'Sector2Time_1_1', 'Sector3Time_1_1',
       'LapPercent_1_1', 'LapPosition_1_1', 'Sector1Percent_1_1',
       'Sector1Position_1_1', 'Sector2Percent_1_1', 'Sector2Position_1_1',
       'Sector3Percent_1_1', 'Sector3Position_1_1', 'LapTime_2',
       'Sector1Time_2', 'Sector2Time_2', 'Sector3Time_2', 'LapPercent_2',
       'LapPosition_2', 'Sector1Percent_2', 'Sector1Position_2',
       'Sector2Percent_2', 'Sector2Position_2', 'Sector3Percent_2',
       'Sector3Position_2', 'LapTime_1_2', 'Sector1Time_1_2',
       'Sector2Time_1_2', 'Sector3Time_1_2', 'LapPercent_1_2',
       'LapPosition_1_2', 'Sector1Percent_1_2', 'Sector1Position_1_2',
       'Sector2Percent_1_2', '

In [45]:
pred_data.to_pickle('pred_data.pkl')